# Récupération des gènes contenant "tail"

In [53]:
import os
import time
import pandas as pd

from Bio import Entrez, SeqIO
from itertools import islice

Liste des "accession number" de chaque phage, fait manuellement parce que c'était trop lent à faire ne python

In [27]:
accession_list = {
    "LN610575.1", "PV268492.1","NC_019918.1", "OQ319934.1", "NC_024140.1",
    "KM434184.1", "ON815903.1", "PP100125.1", "PQ287643.1", "OP342787.1",
    "PQ287643.1", "LN610576.1", "NC_026599.1", "NC_041870.1", "MT119368.1",
    "OQ319934.1", "MT119368.1", "NC_007810.1", "MH536736.1", "MT119368.1",
    "PQ741806.1", "NC_026587.1", "LN610579.1", "NC_019813.1", "LN907801.1",
    "NC_026601.1", "NC_007810.1", "KP340287.1", "MH536736.1", "KP340287.1",
    "PQ676539.1", "PV036882.1", "MT119369.1", "NC_026602.1", "OQ831730.1",
    "NC_010325.1", "OR611941.1", "AB910392.1", "MK803322.1", "MK803322.1",
    "AB910392.1", "CP106742.1"
}

accession_dict = {
    5281:"LN610575.1", 5282:"PV268492.1", 5283:"NC_019918.1", 5284:"OQ319934.1", 5285:"NC_024140.1",
    5286:"KM434184.1", 5287:"ON815903.1", 5288:"PP100125.1", 5289:"PQ287643.1", 5290:"OP342787.1",
    5291:"PQ287643.1", 5292:"LN610576.1", 5293:"NC_026599.1", 5294:"NC_041870.1", 5295:"MT119368.1",
    5296:"OQ319934.1", 5297:"MT119368.1", 5298:"NC_007810.1", 5299:"MH536736.1", 5300:"MT119368.1",
    5301:"PQ741806.1", 5302:"NC_026587.1", 5303:"LN610579.1", 5304:"NC_019813.1", 5305:"LN907801.1",
    5306:"NC_026601.1", 5307:"NC_007810.1", 5308:"KP340287.1", 5309:"MH536736.1", 5310:"KP340287.1",
    5311:"PQ676539.1", 5312:"PV036882.1", 5313:"MT119369.1", 5314:"NC_026602.1", 5315:"OQ831730.1",
    5316:"NC_010325.1", 5317:"OR611941.1", 5318:"AB910392.1", 5319:"MK803322.1", 5320:"MK803322.1",
    5322:"AB910392.1", 5323:"CP106742.1"
}

print(f'longueur avec doublons: {len(accession_list)}')
accession_list = set(accession_list)
print(f'longueur sans doublons: {len(accession_list)}')

longueur avec doublons: 33
longueur sans doublons: 33


In [ ]:
Entrez.email = "michael.strefeler@heig-vd.ch"

def download_genbank(accession, out_dir="data/genbank_files"):
    os.makedirs(out_dir, exist_ok=True)
    filename = os.path.join(out_dir, f"{accession}.gbff")

    if os.path.exists(filename):
        print(f"Already downloaded: {filename}")
        return filename

    try:
        with Entrez.efetch(db="nucleotide", id=accession, rettype="gbwithparts", retmode="text") as handle:
            with open(filename, "w") as out_handle:
                out_handle.write(handle.read())
        time.sleep(0.5)  # Be polite to NCBI
        return filename
    except Exception as e:
        print(f"Failed to download {accession}: {e}")
        return None

In [25]:
for acc in accession_list:
    download_genbank(acc)

Liste des gènes "tail" qu'on trouve dans chaque fichier genbank

In [33]:
gbff_directory = "data/genbank_files"

# Store the accession numbers of matching genes
tail_genes = []

tail_keywords = [
    "tail", "fiber", "baseplate", "sheath", "tube", "tape measure", 
    "portal", "neck", "adapter", "head-to-tail", "connector"
]

# Loop through all .gbff files
for filename in os.listdir(gbff_directory):
    if filename.endswith(".gbff"):
        file_path = os.path.join(gbff_directory, filename)
        
        for record in SeqIO.parse(file_path, "genbank"):
            for feature in record.features:
                if feature.type == "CDS":
                    qualifiers = feature.qualifiers
                    if any(keyword in qualifiers.get(key, [""])[0].lower() 
                           for key in ["product", "note", "protein_id", "gene", "gene_synonym", "function"] if key in qualifiers
                           for keyword in tail_keywords):

                        # Try to extract useful identifiers
                        accession = record.id
                        protein_id = qualifiers.get("protein_id", ["N/A"])[0]
                        product = qualifiers.get("product", ["N/A"])[0]
                        
                        tail_genes.append((accession, protein_id, product))

tail_genes[0:5]

[('LN610579.1', 'CEF89835.1', 'putative tail fiber protein'),
 ('LN610579.1', 'CEF89841.1', 'putative tail fiber protein'),
 ('LN610579.1', 'CEF89846.1', 'putative baseplate protein'),
 ('LN610579.1', 'CEF89849.1', 'putative tail fiber protein'),
 ('LN610579.1', 'CEF89867.1', 'putative tail assembly protein')]

In [ ]:
# Initialize result dictionary
phage_tail_genes = {phage_num: [] for phage_num in accession_dict}

# Reverse lookup: accession to phage number
accession_to_phage = {v: k for k, v in accession_dict.items()}

for filename in os.listdir(gbff_directory):
    if filename.endswith(".gbff"):
        file_path = os.path.join(gbff_directory, filename)
        
        for record in SeqIO.parse(file_path, "genbank"):
            accession = record.id
            phage_num = accession_to_phage.get(accession)

            if phage_num is None:
                continue  # skip files not in your accession_dict

            for feature in record.features:
                if feature.type == "CDS":
                    qualifiers = feature.qualifiers

                    # Look for 'tail' in various qualifiers
                    if any("tail" in qualifiers.get(key, [""])[0].lower() 
                           for key in ["product", "note", "protein_id", "gene", "gene_synonym", "function"] 
                           if key in qualifiers):

                        protein_id = qualifiers.get("protein_id", ["N/A"])[0]
                        product = qualifiers.get("product", ["N/A"])[0]

                        phage_tail_genes[phage_num].append((protein_id, product))
                        
dict(islice(phage_tail_genes.items(), 0, 5))

{5281: [('CEF89381.1', 'putative tail length tape-measure protein'),
  ('CEF89391.1', 'putative tail fiber protein')],
 5282: [],
 5283: [('YP_007236842.1',
   'RNA ligase and tail fiber protein attachment catalyst'),
  ('YP_007236878.1', 'tail sheath'),
  ('YP_007236882.1', 'tail assembly chaperone'),
  ('YP_007236883.1', 'tail length tape measure protein'),
  ('YP_007236884.1', 'tail fiber protein'),
  ('YP_007236892.1', 'tail fiber protein'),
  ('YP_007236893.1', 'tail fiber assembly'),
  ('YP_007236894.1', 'tail fiber protein')],
 5284: [],
 5285: [('YP_009031825.1', 'tail fiber protein'),
  ('YP_009031828.1', 'tail fiber protein'),
  ('YP_009031850.1', 'tail length tape measure protein')]}

Maintenant qu'on a la liste des gènes "tail" par phage on doit...

In [61]:
phage_tail_genes.get(5305)

[('CUT98408.1', 'putative head-tail connector protein'),
 ('CUT98414.1', 'putative tail tape measure protein'),
 ('CUT98421.1', 'putative tail protein')]

In [62]:
phage_tail_genes.get(5306)

[('YP_009125674.1', 'tail completion or Neck1 protein'),
 ('YP_009125683.1', 'head-tail adaptor'),
 ('YP_009125686.1', 'major tail protein with Ig-like domain'),
 ('YP_009125687.1', 'tail assembly chaperone'),
 ('YP_009125688.1', 'tail length tape measure protein'),
 ('YP_009125691.1', 'tail assembly protein'),
 ('YP_009125692.1', 'tail assembly protein'),
 ('YP_009125693.1', 'tail assembly chaperone'),
 ('YP_009125694.1', 'tail assembly chaperone'),
 ('YP_009125695.1', 'tail protein')]

In [ ]:
df = pd.read_excel("data/InteractionsPhages_PseudomonasAeruginosa_Revised.xlsx")
interaction_matrix = pd.crosstab(df['bacterium_id'], df['bacteriophage_id'])